In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import *

def stringmapper(s: str) -> torch.Tensor:
    return torch.tensor([x for x in s.encode()])

print(stringmapper("hello").sum())

#TODO actually load real weights from a generated model
class ALSModel(nn.Module):
    def __init__(self, n_items: int, n_factors: int):
        super(ALSModel, self).__init__()
        #TODO add user --> item
        self.emb = nn.Embedding(n_items, n_factors)
        self.it2ind = {str(i): i for i in range(n_items)}# TODO use real values
        self.ind2it = {v:k for k,v in self.it2ind.items()}
        
    def map_out(self, scores: torch.Tensor, indices: torch.Tensor) -> Dict[str, float]:
        inds: List[int] = indices.squeeze().tolist()
        scorez: List[float] = scores.squeeze().tolist()
        return {self.ind2it[i]: s for i, s in zip(inds, scorez)}
    
    def forward(self, ind: torch.Tensor, size: int = 30)-> Dict[str, float]:
    #def forward(self, ind: torch.Tensor)-> torch.Tensor:
    #def forward(self, ind: torch.Tensor, size: int = 5)-> Tuple[torch.Tensor, torch.Tensor]:
    #def forward(self, ind: torch.Tensor, size: int = 5)-> str:#List[torch.Tensor]:
        #ind = torch.tensor([self.it2ind[inp]]).long()
        u = self.emb(ind)
        scores = u @ self.emb.weight.t()
        s, i = scores.topk(size)
        #return self.map_out(s, i)
        #return s.squeeze(), i.squeeze()
        return [self.ind2it.get(x, "unk") for x in i.squeeze()]
        
        
n_items, n_factors = (10000, 128)     
m = ALSModel(n_items, n_factors)
m2 = torch.jit.script(m)
    
#TODO can we actually map strings and stuff in ONNX? It seems like we cant???
#m("5", size=5), m2("5", size=5)


tensor(532)


RuntimeError: 
Expected an annotation of type List but got Dict[str, float]:
  File "/tmp/ipykernel_50175/2170355860.py", line 35
        #return self.map_out(s, i)
        #return s.squeeze(), i.squeeze()
        return [self.ind2it.get(x, "unk") for x in i.squeeze()]
               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE


In [2]:
o = m2(torch.ones(1).long())
o#, o.sort(descending=True)

['1', '3089', '2481', '7414', '9328']

In [17]:
#x = (torch.ones(1).long())
x = (torch.ones(1).long(), 3)
file_path = "data/model2.onnx"
# Strange behavior, dynamic axes is an array of ints if unnamed node is used, but dict if its named....
# https://pytorch.org/docs/stable/onnx.html

output_names = ['scores', 'indices']
dynamic_axes = {name: {0: f'{name}_dyn_axis'} for name in output_names}
output_names, dynamic_axes = None, None
torch.onnx.export(m2, x, file_path, input_names=['contentId', 'size'], output_names=output_names, dynamic_axes=dynamic_axes, verbose=True, opset_version=16)

Torch IR graph at exception: graph(%ind : Long(1, strides=[1], requires_grad=0, device=cpu),
      %size : Long(device=cpu)):
  %3 : str = prim::Constant[value="heyo"]() # /tmp/ipykernel_50175/1157316797.py:35:15
  return (%3)



RuntimeError: unexpected tensor scalar type

In [ ]:
1+1

In [100]:
import onnx
# ensure a valid onnx file
onnx.checker.check_model(onnx.load(file_path))

In [101]:
import onnxruntime
import numpy as np

# sanity test outputs
ort_session = onnxruntime.InferenceSession(file_path)
inp = np.ones(1, dtype=np.int64) * 15
ort_inputs = {ort_session.get_inputs()[0].name: inp, ort_session.get_inputs()[1].name: inp}
ort_outs = ort_session.run(None, ort_inputs)

print(ort_outs)

[array([140.76741 ,  47.154198,  46.158905,  43.749146,  43.0817  ,
        40.787643,  40.081074,  39.618313,  39.11862 ,  38.988125,
        38.269043,  36.655247,  36.400436,  36.3293  ,  35.509876],
      dtype=float32), array([  15, 6544, 2713, 6677, 6579, 4016, 1982, 8540, 9330, 9086, 8393,
       1176, 5090, 3117, 6752], dtype=int64)]


In [91]:
ort_session.get_inputs()[0].name

'contentId'

In [ ]:
class ModelWrapper:
    #TODO named lookups
    #inspect names of inputs in function, name onnx nodes accordingly (?).
    # "magically" map self.${X}_it2ind -> a json file which wraps all inputs on node named ${X}? file called: ${X}.json
    
    # "MappedInput(it2ind, f(x)) where f(x) could be Seq(items) --> Tensor[...] or Option[Item] --> Tensor[] for example?
    
    #TODO output contract of models would need to be something like
    # scores: Tensor[float], model_indices: Tensor[Long], which could be mapped afterwards to a map of scores... (?)
    
    
    
    #TODO the big question::::: How export jsons easily of all map-dicts? bonus points for making it easily usable in Python at the same time.
    
    
    

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import *

class ALSModel(nn.Module):
    def __init__(self, n_items: int, n_factors: int):
        super(ALSModel, self).__init__()

    def forward(self, ind: torch.Tensor, size: int = 5)-> str:
        return "heyo"
        
        
m = torch.jit.script(ALSModel(10000, 128))
x = (torch.ones(1).long(), 3)

torch.onnx.export(m, x, "data/model2.onnx", input_names=['contentId', 'size'],  verbose=True, opset_version=16)

Torch IR graph at exception: graph(%ind : Long(1, strides=[1], requires_grad=0, device=cpu),
      %size : Long(device=cpu)):
  %3 : str = prim::Constant[value="heyo"]() # /tmp/ipykernel_53998/1479605971.py:11:15
  return (%3)



RuntimeError: unexpected tensor scalar type

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import *

class ALSModel(nn.Module):
    def __init__(self, n_items: int, n_factors: int):
        super(ALSModel, self).__init__()
        self.emb = nn.Embedding(n_items, n_factors)
        self.it2ind = {str(i): i for i in range(n_items)}# TODO use real values
        self.ind2it = {v:k for k,v in self.it2ind.items()}
        
    def map_out(self, scores: torch.Tensor, indices: torch.Tensor) -> Dict[str, float]:
        inds: List[int] = indices.squeeze().tolist()
        scorez: List[float] = scores.squeeze().tolist()
        return {self.ind2it.get(i, "a"): s for i, s in zip(inds, scorez)}
        
    def forward(self, ind: torch.Tensor, size: int = 30)-> Dict[str, float]:
        u = self.emb(ind)
        scores = u @ self.emb.weight.t()
        s, i = scores.topk(size)
        return self.map_out(s, i)
    
m = torch.jit.script(ALSModel(10000, 128))
x = (torch.ones(1).long(), 3)

torch.onnx.export(m, x, "data/model2.onnx", input_names=['contentId', 'size'],  verbose=True, opset_version=16)